## Day 24 - Colliding hailstones

**Part 1: Disregarding z-axis, how many hailstones will collide between with an X and Y position each at least 200000000000000 and at most 400000000000000.**

In [7]:
with open("./example.txt") as f:
    example_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

example_lines

['19, 13, 30 @ -2,  1, -2',
 '18, 19, 22 @ -1, -1, -2',
 '20, 25, 34 @ -2, -2, -4',
 '12, 31, 28 @ -1, -2, -1',
 '20, 19, 15 @  1, -5, -3']

In [8]:
def intersection_time1(x1, x2, y1, y2, vx1, vx2, vy1, vy2) -> float:
    return (vx2*(y2-y1)+vy1*(x1-x2))/(vx2*vy1 - vx1*vy2)

In [9]:
def intersection_time2(x1, x2, y1, y2, vx1, vx2, vy1, vy2) -> float:
    return (y1 - y2 + (vy1/vx1)*(x2-x1))/(vy2 - (vx2/vx1))

In [10]:
def position_at_time1(x1, y1, vx1, vy1, t1):
    return (x1 + vx1*t1, y1 + vy1*t1)

In [11]:
def position_at_time2(x2, y2, vx2, vy2, t2):
    return (x2 + vx2*t2, y2 + vy2*t2)

In [35]:
from itertools import combinations

def part1(lines: list[str], example: bool = False) -> int:
    lower_bound = 7 if example else 200000000000000
    upper_bound = 27 if example else 400000000000000

    hailstones = [
        tuple(map(int,
                  (line.split(",")[0], line.split(", ")[1], line.split("@ ")[-1].split(",")[0], line.split("@ ")[-1].split(", ")[1])
                  ))
        for line in lines
    ]  # (X, Y, vX, vY)

    found = 0

    for (x1, y1, vx1, vy1), (x2, y2, vx2, vy2) in combinations(hailstones, r=2):
        print()
        try:
            t1 = intersection_time1(x1, x2, y1, y2, vx1, vx2, vy1, vy2)
            t2 = intersection_time2(x1, x2, y1, y2, vx1, vx2, vy1, vy2)
            print(f"{x1=} {y1=} {x2=} {y2=} {vx1=} {vy1=} {vx2=} {vy2=}")
        except:
            print(f"no {x1=} {y1=} {x2=} {y2=}")
            continue
        xval1, yval1 = position_at_time1(x1, y1, vx1, vy1, t1)
        xval2, yval2 = position_at_time2(x2, y2, vx2, vy2, t2)
        if (
            # first intersection valid
            lower_bound <= xval1 <= upper_bound
            and lower_bound <= yval1 <= upper_bound
            and t1 >= 0
            # second intersection valid
            and lower_bound <= xval2 <= upper_bound
            and lower_bound <= yval2 <= upper_bound
            and t2 >= 0
        ):
            print(f"Within! {xval1= } {yval1=} {t1=} | {xval2= } {yval2=} {t2=}")
            found += 1
        else:
            pass
            print(f"OUtside! {xval1= } {yval1=} {t1=} | {xval2= } {yval2=} {t2=}")

    return found

        

assert part1(example_lines, example=True) == 2
# part1(input_lines, example=False)  # 4152 is too low



x1=19 y1=13 x2=18 y2=19 vx1=-2 vy1=1 vx2=-1 vy2=-1
Within! xval1= 15.666666666666666 yval1=14.666666666666666 t1=1.6666666666666667 | xval2= 14.333333333333334 yval2=15.333333333333334 t2=3.6666666666666665

x1=19 y1=13 x2=20 y2=25 vx1=-2 vy1=1 vx2=-2 vy2=-2
Within! xval1= 10.666666666666666 yval1=17.166666666666668 t1=4.166666666666667 | xval2= 11.666666666666666 yval2=16.666666666666664 t2=4.166666666666667

x1=19 y1=13 x2=12 y2=31 vx1=-2 vy1=1 vx2=-1 vy2=-2
OUtside! xval1= 14.6 yval1=15.2 t1=2.2 | xval2= 6.2 yval2=19.4 t2=5.8

x1=19 y1=13 x2=20 y2=19 vx1=-2 vy1=1 vx2=1 vy2=-5
OUtside! xval1= 20.11111111111111 yval1=12.444444444444445 t1=-0.5555555555555556 | xval2= 21.444444444444443 yval2=11.777777777777779 t2=1.4444444444444444

no x1=18 y1=19 x2=20 y2=25

x1=18 y1=19 x2=12 y2=31 vx1=-1 vy1=-1 vx2=-1 vy2=-2
OUtside! xval1= 0.0 yval1=1.0 t1=18.0 | xval2= 6.0 yval2=19.0 t2=6.0

x1=18 y1=19 x2=20 y2=19 vx1=-1 vy1=-1 vx2=1 vy2=-5
OUtside! xval1= 18.333333333333332 yval1=19.3333333333

In [27]:
class Hailstone:
    def __init__(self, sx, sy, sz, vx, vy, vz):
        self.sx = sx
        self.sy = sy
        self.sz = sz
        self.vx = vx
        self.vy = vy
        self.vz = vz
        
        self.a = vy
        self.b = -vx
        self.c = vy * sx - vx * sy
    
    def __repr__(self):
        return "Hailstone{" + f"a={self.a}, b={self.b}, c={self.c}" + "}"

hailstones = [Hailstone(*map(int, line.replace("@", ",").split(","))) for line in input_lines]

total = 0

for i, hs1 in enumerate(hailstones):
    for hs2 in hailstones[:i]:
        a1, b1, c1 = hs1.a, hs1.b, hs1.c
        a2, b2, c2 = hs2.a, hs2.b, hs2.c
        if a1 * b2 == b1 * a2:
            continue
        x = (c1 * b2 - c2 * b1) / (a1 * b2 - a2 * b1)
        y = (c2 * a1 - c1 * a2) / (a1 * b2 - a2 * b1)
        if 200000000000000 <= x <= 400000000000000 and 200000000000000 <= y <= 400000000000000:
            if all((x - hs.sx) * hs.vx >= 0 and (y - hs.sy) * hs.vy >= 0 for hs in (hs1, hs2)):
                total += 1

print(total)

14799
